In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
df = pd.read_csv('obesity.csv')

In [3]:
df

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,Female,21.000000,1.620000,64.000000,yes,no,2.0,3.0,Sometimes,no,2.000000,no,0.000000,1.000000,no,Public_Transportation,Normal_Weight
1,Female,21.000000,1.520000,56.000000,yes,no,3.0,3.0,Sometimes,yes,3.000000,yes,3.000000,0.000000,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.000000,1.800000,77.000000,yes,no,2.0,3.0,Sometimes,no,2.000000,no,2.000000,1.000000,Frequently,Public_Transportation,Normal_Weight
3,Male,27.000000,1.800000,87.000000,no,no,3.0,3.0,Sometimes,no,2.000000,no,2.000000,0.000000,Frequently,Walking,Overweight_Level_I
4,Male,22.000000,1.780000,89.800000,no,no,2.0,1.0,Sometimes,no,2.000000,no,0.000000,0.000000,Sometimes,Public_Transportation,Overweight_Level_II
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2106,Female,20.976842,1.710730,131.408528,yes,yes,3.0,3.0,Sometimes,no,1.728139,no,1.676269,0.906247,Sometimes,Public_Transportation,Obesity_Type_III
2107,Female,21.982942,1.748584,133.742943,yes,yes,3.0,3.0,Sometimes,no,2.005130,no,1.341390,0.599270,Sometimes,Public_Transportation,Obesity_Type_III
2108,Female,22.524036,1.752206,133.689352,yes,yes,3.0,3.0,Sometimes,no,2.054193,no,1.414209,0.646288,Sometimes,Public_Transportation,Obesity_Type_III
2109,Female,24.361936,1.739450,133.346641,yes,yes,3.0,3.0,Sometimes,no,2.852339,no,1.139107,0.586035,Sometimes,Public_Transportation,Obesity_Type_III


In [4]:
df['Gender'] = df['Gender'].replace({'Male':0,'Female':1})
df['family_history_with_overweight'] = df['family_history_with_overweight'].replace({'yes':1,'no':0})
df['SMOKE'] = df['SMOKE'].replace({'no':0,'yes':1})

In [5]:
df

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,1,21.000000,1.620000,64.000000,1,no,2.0,3.0,Sometimes,0,2.000000,no,0.000000,1.000000,no,Public_Transportation,Normal_Weight
1,1,21.000000,1.520000,56.000000,1,no,3.0,3.0,Sometimes,1,3.000000,yes,3.000000,0.000000,Sometimes,Public_Transportation,Normal_Weight
2,0,23.000000,1.800000,77.000000,1,no,2.0,3.0,Sometimes,0,2.000000,no,2.000000,1.000000,Frequently,Public_Transportation,Normal_Weight
3,0,27.000000,1.800000,87.000000,0,no,3.0,3.0,Sometimes,0,2.000000,no,2.000000,0.000000,Frequently,Walking,Overweight_Level_I
4,0,22.000000,1.780000,89.800000,0,no,2.0,1.0,Sometimes,0,2.000000,no,0.000000,0.000000,Sometimes,Public_Transportation,Overweight_Level_II
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2106,1,20.976842,1.710730,131.408528,1,yes,3.0,3.0,Sometimes,0,1.728139,no,1.676269,0.906247,Sometimes,Public_Transportation,Obesity_Type_III
2107,1,21.982942,1.748584,133.742943,1,yes,3.0,3.0,Sometimes,0,2.005130,no,1.341390,0.599270,Sometimes,Public_Transportation,Obesity_Type_III
2108,1,22.524036,1.752206,133.689352,1,yes,3.0,3.0,Sometimes,0,2.054193,no,1.414209,0.646288,Sometimes,Public_Transportation,Obesity_Type_III
2109,1,24.361936,1.739450,133.346641,1,yes,3.0,3.0,Sometimes,0,2.852339,no,1.139107,0.586035,Sometimes,Public_Transportation,Obesity_Type_III


In [6]:
df.columns

Index(['Gender', 'Age', 'Height', 'Weight', 'family_history_with_overweight',
       'FAVC', 'FCVC', 'NCP', 'CAEC', 'SMOKE', 'CH2O', 'SCC', 'FAF', 'TUE',
       'CALC', 'MTRANS', 'NObeyesdad'],
      dtype='object')

In [7]:
df = df[['Gender', 'Age', 'Height', 'Weight', 'family_history_with_overweight','SMOKE','NObeyesdad']]
df

,Gender,Age,Height,Weight,family_history_with_overweight,SMOKE,NObeyesdad
0,1,21.000000,1.620000,64.000000,1,0,Normal_Weight
1,1,21.000000,1.520000,56.000000,1,1,Normal_Weight
2,0,23.000000,1.800000,77.000000,1,0,Normal_Weight
3,0,27.000000,1.800000,87.000000,0,0,Overweight_Level_I
4,0,22.000000,1.780000,89.800000,0,0,Overweight_Level_II
...,...,...,...,...,...,...,...
2106,1,20.976842,1.710730,131.408528,1,0,Obesity_Type_III
2107,1,21.982942,1.748584,133.742943,1,0,Obesity_Type_III
2108,1,22.524036,1.752206,133.689352,1,0,Obesity_Type_III
2109,1,24.361936,1.739450,133.346641,1,0,Obesity_Type_III


In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X = df.drop('NObeyesdad', axis=1)
Y = df['NObeyesdad']



In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size =0.2, shuffle=True, random_state=42, 
                                                    stratify=Y)

In [11]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

model = Pipeline([
    ('scaler', MinMaxScaler()),
    ('knn', KNeighborsClassifier(n_neighbors=5))
])
model.fit(X_train, y_train)
y_pred_test = model.predict(X_test)
report = classification_report(y_test, y_pred_test, digits=4)
print(report)

                     precision    recall  f1-score   support

Insufficient_Weight     0.9298    0.9815    0.9550        54
      Normal_Weight     0.8936    0.7241    0.8000        58
     Obesity_Type_I     0.8767    0.9143    0.8951        70
    Obesity_Type_II     0.9516    0.9833    0.9672        60
   Obesity_Type_III     1.0000    0.9846    0.9922        65
 Overweight_Level_I     0.7903    0.8448    0.8167        58
Overweight_Level_II     0.7931    0.7931    0.7931        58

           accuracy                         0.8913       423
          macro avg     0.8907    0.8894    0.8885       423
       weighted avg     0.8921    0.8913    0.8901       423



In [12]:
import pickle
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

In [13]:
model.predict(X_train)

array(['Overweight_Level_I', 'Normal_Weight', 'Insufficient_Weight', ...,
       'Insufficient_Weight', 'Obesity_Type_II', 'Normal_Weight'],
      dtype=object)